In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import json

import os
os.chdir('..')


from src.swbm import *
from src.plots import *
from src.utils import *

In [2]:
# Load data and format
input_swbm_ger = pd.read_csv('data/Data_swbm_Germany.csv')
input_ger = prepro(input_swbm_ger)

input_swbm_swe = pd.read_csv('data/Data_swbm_Sweden.csv')
input_swe = prepro(input_swbm_swe)

input_swbm_esp = pd.read_csv('data/Data_swbm_Spain.csv')
input_esp = prepro(input_swbm_esp)

sites = [input_ger, input_swe, input_esp]

In [3]:
# Load best parameters
with open('results/ger_output.json', 'r') as file:
    ger_calib = json.load(file)

with open('results/swe_output.json', 'r') as file:
    swe_calib = json.load(file)

with open('results/esp_output.json', 'r') as file:
    esp_calib = json.load(file)

calibs = [ger_calib, swe_calib, esp_calib]
countries = ['Germany', 'Sweden', 'Spain']

In [4]:
# Create test masks
start_test = '2013-01-01'
end_test = '2018-12-31'
test_mask = (input_ger['time'] >= start_test) & (input_ger['time'] <= end_test)

start_eval = '2014-01-01'
end_eval = end_test

In [5]:
# test SWBM for all sites
swbm_param = 'b0'
evals_test = []
model_results = []
for site, calib, country in zip(sites, calibs, countries):
    # get test data
    input_swbm = site[test_mask]
    input_swbm.reset_index(drop=True, inplace=True)

    # get eval mask and eval input
    eval_mask = (input_swbm['time'] >= start_eval) & (input_swbm['time'] <= end_eval)
    input_eval = input_swbm[eval_mask]
    input_eval.reset_index(drop=True, inplace=True)

    const_swbm_params = calib[0].copy()
    params_seasonal = calib[0].copy()

    # Get sinus curve for current single parameter
    if country == 'Sweden':
        params_seasonal[swbm_param] = seasonal_sinus(
            len(input_swbm),
            amplitude=calib[3]['amplitude'] * 200,
            freq=calib[3]['freq'],
            phase=calib[3]['phase'],
            center=calib[3]['center'] * 200,
            which=swbm_param)
        params_seasonal['c_s'] = 420
    else:
        params_seasonal[swbm_param] = seasonal_sinus(
            len(input_swbm),
            amplitude=calib[3]['amplitude'],
            freq=calib[3]['freq'],
            phase=calib[3]['phase'],
            center=calib[3]['center'],
            which=swbm_param) 
    
    # Run SWBM with optimized seasonal b0
    preds_seasonal = predict_ts(input_swbm, params_seasonal)
    moists_seasonal, runoffs_seasonal, ets_seasonal, na_count = preds_seasonal

    # Save data in dataframe
    model_result = input_swbm.copy()
    model_result = pd.concat([model_result, pd.DataFrame({'moists_seasonal': moists_seasonal,
                                  'runoffs_seasonal': runoffs_seasonal,
                                  'ets_seasonal': ets_seasonal}, index=model_result.index)], axis=1)

    # cut data for evaluation
    moists_eval = moists_seasonal[eval_mask]
    runoffs_eval = runoffs_seasonal[eval_mask]
    ets_eval = ets_seasonal[eval_mask]
    
    # calculate correlation
    eval_df = eval_swbm(input_eval,
                        {'sm': moists_eval,
                        'ro': runoffs_eval,
                        'le': ets_eval},
                        swbm_param)
    
    result_test = {
                'Combination': calib[0],
                'sum_corr': eval_df['corr'].sum(),
                'SinusParameters': calib[3],
                'eval_df': eval_df,
            }

    evals_test.append(result_test)
    model_results.append(model_result)

In [6]:
evals_test

[{'Combination': {'c_s': 420, 'b0': 0.4, 'g': 0.2, 'a': 8},
  'sum_corr': 2.525792435676959,
  'SinusParameters': {'amplitude': 0.15773671537646436,
   'freq': 1.991013217918756,
   'phase': 4.695383716350022,
   'center': 0.26580671413344364},
  'eval_df':   parameter kind      corr           pval
  0        b0   sm  0.903367   0.000000e+00
  1        b0   ro  0.696029  1.034909e-264
  2        b0   le  0.926397   0.000000e+00},
 {'Combination': {'c_s': 840, 'b0': 0.4, 'g': 0.5, 'a': 4},
  'sum_corr': 1.2218900864420017,
  'SinusParameters': {'amplitude': 0.0005993492667837831,
   'freq': 1.9982300430160138,
   'phase': 4.113755963808566,
   'center': 0.0008309403292962054},
  'eval_df':   parameter kind      corr          pval
  0        b0   sm  0.226403  1.178010e-22
  1        b0   ro  0.072256  2.004698e-03
  2        b0   le  0.923231  0.000000e+00},
 {'Combination': {'c_s': 210, 'b0': 0.4, 'g': 0.8, 'a': 8},
  'sum_corr': 2.530976745071012,
  'SinusParameters': {'amplitude': 0.

In [7]:
# Transform evaluation
ger_test_evals = transform_evals(evals_test[0])
swe_test_evals = transform_evals(evals_test[1])
esp_test_evals = transform_evals(evals_test[2])

In [8]:
# Save evaluations
with open('results/ger_test_eval.json', 'w') as file:
    json.dump(ger_test_evals, file)

with open('results/swe_test_eval.json', 'w') as file:
    json.dump(swe_test_evals, file)

with open('results/esp_test_eval.json', 'w') as file:
    json.dump(esp_test_evals, file)

In [9]:
# Save model predictions
model_results[0].to_csv('data/output/ger_model_test.csv', index=False)
model_results[1].to_csv('data/output/swe_model_test.csv', index=False)
model_results[2].to_csv('data/output/esp_model_test.csv', index=False)